# Coastlines 

* **Products used:** [DE Africa Coastlines](https://docs.digitalearthafrica.org/en/latest/data_specs/Coastlines_specs.html)


## Background

Africa's long and dynamic coastline is subject to a wide range of pressures, including extreme weather and climate, sea level rise and human development.
Understanding how the coastline responds to these pressures is crucial to managing this region, from social, environmental and economic perspectives. 

The [Digital Earth Africa Coastlines](https://docs.digitalearthafrica.org/en/latest/data_specs/Coastlines_specs.html) is a continental dataset that includes annual shorelines and rates of coastal change along the entire African coastline from 2000 to 2021. 

The product combines satellite data from the Digital Earth Africa program with tidal modelling to map the typical location of the coastline at mean sea level for each year. 
The product enables trends of coastal erosion and growth to be examined annually at both a local and continental scale, and for patterns of coastal change to be mapped historically and updated regularly as data continues to be acquired. 
This allows current rates of coastal change to be compared with that observed in previous years or decades. 

The ability to map shoreline positions for each year provides valuable insights into whether changes to the coastline are the result of particular events or actions, or a process of more gradual change over time. 
This information can enable scientists, managers and policy makers to assess impacts from the range of drivers impacting the coastlines and potentially assist planning and forecasting for future scenarios. 

### Applications
* Monitoring and mapping rates of coastal erosion along the African coastline 
* Prioritise and evaluate the impacts of local and regional coastal management based on historical coastline change 
* Modelling how coastlines respond to drivers of change, including extreme weather events, sea level rise or human development 
* Supporting geomorphological studies of how and why coastlines have changed across time 

For a full description of the product specifications, validation results, and methods used to develop the products, see the [DE Africa Coastlines specifications](https://docs.digitalearthafrica.org/en/latest/data_specs/Coastlines_specs.html) document.

## Description

This notebook will demonstrate how to load, plot, and conduct a simple analysis using the [Digital Earth Africa Coastlines](https://docs.digitalearthafrica.org/en/latest/data_specs/Coastlines_specs.html) product.
 
Topics covered include:

1. Loading DE Africa Coastlines annual shoreline data using the `get_coastlines` function.
1. Loading DE Africa Coastlines rates of change statistics data using the `get_coastlines` function.
2. Extracting shoreline data for one or multiple coastal transects using an interactive DE Africa Coastlines application.

***

## Getting started

To run this analysis, run all the cells in the notebook, starting with the "Load packages" cell.

### Load packages
Import Python packages that are used for the analysis.

In [1]:
# Force GeoPandas to use Shapely instead of PyGEOS
# In a future release, GeoPandas will switch to using Shapely by default.
import os
os.environ['USE_PYGEOS'] = '0'

import geopandas as gpd
import matplotlib.pyplot as plt

from deafrica_tools.coastal import get_coastlines
from deafrica_tools.app import deacoastlines
from deafrica_tools.plotting import display_map

### Analysis parameters

This section defines the analysis parameters, including:

* `central_lat, central_lon, buffer`: center lat/lon and analysis window size for the area of interest

The default location is an area on Nyali Beach along the Kenyan Coast.

In [2]:
# Set the central latitude and longitude.
central_lat = -4.049068
central_lon = 39.708946

# Set the buffer to load around the central coordinates.
buffer = 0.03

# Compute the bounding box for the study area. 
xmin, xmax = central_lon - buffer, central_lon + buffer
ymin, ymax = central_lat - buffer, central_lat + buffer
bbox = (xmin, ymin, xmax, ymax)

## View the selected location

The next cell will display the selected area on an interactive map. Feel free to zoom in and out to get a better understanding of the area you'll be analysing. Clicking on any point of the map will reveal the latitude and longitude coordinates of that point.

In [3]:
display_map(x=(xmin, xmax), y=(ymin, ymax))

## Loading DE Africa Coastlines data 

### Annual shorelines

DE Africa Coastlines data can be loaded directly in a Python script or Jupyter Notebook using the `get_coastlines` function. This function makes use of the DE Africa Coastlines Web Feature Service (WFS) and `geopandas`to load the DE Africa Coastlines data for a provided bounding box.

In [4]:
# Load DE Africa Coastlines annual shorelines data. 
deacl_shorelines_gdf = get_coastlines(bbox=bbox, 
                                      layer='shorelines')

# Print example output.
deacl_shorelines_gdf.head()

,id,year,certainty,tide_datum,country,geometry
15,coastlines_v0_4_0_shorelines_annual.7926,2012,good,0 m AMSL,Kenya,"MULTILINESTRING ((39.68133 -4.05546, 39.68144 ..."
27,coastlines_v0_4_0_shorelines_annual.7942,2021,good,0 m AMSL,Kenya,"MULTILINESTRING ((39.68134 -4.05546, 39.68144 ..."
23,coastlines_v0_4_0_shorelines_annual.7938,2019,good,0 m AMSL,Kenya,"MULTILINESTRING ((39.68131 -4.05546, 39.68144 ..."
14,coastlines_v0_4_0_shorelines_annual.7925,2011,good,0 m AMSL,Kenya,"MULTILINESTRING ((39.68134 -4.05546, 39.68144 ..."
2,coastlines_v0_4_0_shorelines_annual.7910,2002,good,0 m AMSL,Kenya,"MULTILINESTRING ((39.68135 -4.05546, 39.68144 ..."


We can now plot the DE Africa Coastlines annual shoreline data using `geopandas`. 
Zoom in to **Nyali Beach** in Kenya on the interactive map to view the shoreline data.

Annual shorelines represent the median or 'most representative' position of the shoreline at approximately mean sea level tide (0 m Above Mean Sea Level) for each year from 2000 onward.
Light-coloured lines (e.g. yellow) in the plot below represent recent shorelines (e.g. 2020), while dark-coloured lines represent older shorelines (e.g. 2000).

> **Note:** For more detail about DE Africa Coastlines annual shoreline data, refer to the official [Digital Earth Africa Coastlines product description](https://docs.digitalearthafrica.org/en/latest/data_specs/Coastlines_specs.html).

In [5]:
# Plot on interactive map of DE Africa Coastlines data within our bounding box.
deacl_shorelines_gdf.explore(column='year', cmap='inferno', categorical=True)

### Rates of change statistics points

We can also load and plot the DE Africa Coastlines rates of change statistics points.
These points provide robust rates of annual coastal change (in metres per year) for every 30 m along Africa's non-rocky (e.g. sandy and muddy) coastlines. 
These rates are calculated by linearly regressing annual shoreline positions against time, using the most recent shoreline as a baseline.

Negative values (red points) indicate retreat (e.g. erosion), and positive values indicate growth (e.g. progradation) over time.
By default, rates of change are shown for points with a statistically significant trend over time only.

> **Note:** For more detail about DE Africa Coastlines rates of change statistics data, refer to the official [Digital Earth Africa Coastlines product description](https://docs.digitalearthafrica.org/en/latest/data_specs/Coastlines_specs.html).

In [6]:
# Load DE Africa Coastlines annual shorelines data within our bounding box.
deacl_statistics_gdf = get_coastlines(bbox=(xmin, ymin, xmax, ymax),
                                      layer='statistics')

# Filter points to statistically significant results only.
deacl_statistics_gdf = deacl_statistics_gdf.loc[
    deacl_statistics_gdf.sig_time < 0.01]

# Plot on interactive map.
deacl_statistics_gdf.explore(column='rate_time',
                             cmap='RdBu',
                             vmin=-3,
                             vmax=3,
                             tooltip='rate_time')

The DE Africa Coastlines rates of change statistics points data also provides many additional statistics that give insights into coastal change in Africa.
For a full description of each of these statistics, refer to the Rates of change statistics section of the official [Digital Earth Africa Coastlines product description](https://docs.digitalearthafrica.org/en/latest/data_specs/Coastlines_specs.html).

In [7]:
deacl_statistics_gdf.head()

,id,uid,rate_time,sig_time,se_time,outl_time,dist_2000,dist_2001,dist_2002,dist_2003,...,angle_std,valid_obs,valid_span,sce,nsm,max_year,min_year,certainty,country,geometry
297,coastlines_v0_4_0_rates_of_change.328139,None,0.33,0.001,0.09,,-3.74,-6.38,-8.48,-2.94,...,6,22,22,12.76,3.74,2018,2002,good,Kenya,POINT (39.67895 -4.07368)
299,coastlines_v0_4_0_rates_of_change.328141,None,0.23,0.008,0.08,,-3.47,-5.78,-5.89,-4.00,...,11,22,22,10.17,3.47,2014,2007,good,Kenya,POINT (39.67943 -4.07347)
311,coastlines_v0_4_0_rates_of_change.328153,None,0.43,0.000,0.09,2006,-4.94,-6.30,-7.58,-9.34,...,7,21,22,12.74,4.94,2020,2003,good,Kenya,POINT (39.68194 -4.07201)
328,coastlines_v0_4_0_rates_of_change.328170,None,0.20,0.008,0.07,2005,-7.17,-7.00,-7.74,-8.24,...,11,21,22,8.72,7.17,2021,2004,good,Kenya,POINT (39.68211 -4.06809)
58,coastlines_v0_4_0_rates_of_change.327500,None,0.49,0.003,0.14,,-6.10,-7.28,-7.83,-8.54,...,6,22,22,16.39,6.10,2014,2003,good,Kenya,POINT (39.69146 -4.06377)


### Exporting DE Africa Coastlines data as vector files
We can easily export the loaded data as spatial vector files (e.g. ESRI Shapefiles or GeoJSON) so that they can be analysed further in GIS software:

In [8]:
deacl_shorelines_gdf.to_file('deacoastlines_shorelines.shp')
deacl_statistics_gdf.to_file('deacoastlines_statistics.shp')

## DE Africa Coastlines analysis tools

The following sections provide useful tools for analysing DE Africa Coastlines data directly within a Jupyter Notebook without needing to load the data into GIS software.
All outputs from the tools below will be saved to a new folder in this directory called `deacoastlines_outputs`.

### Extract shoreline data by coastal transect
The `deacoastlines.transect_app` tool allows you to interactively draw a transect over DE Africa Coastlines annual shoreline data, and get back a graph showing how shoreline positions have changed over time.
To use the tool:

1. Run the cell below; an interactive application will appear 
2. Zoom in on the map and use the `Draw a polyline` tool on the left to draw a transect so that it crosses through a set of shorelines (see example below). Transects should be drawn in a consistent direction, e.g. starting on land and finishing over water. This ensures that results can be compared between transects.

![Transect example](../Supplementary_data/Coastlines/deacoastlines_transect.gif)

3. Press `Finish` when you are happy with the line, then click on the `Extract shoreline data` button on the bottom left.
4. A graph will appear below the map showing distances along the transect to each annual shoreline (distances will be measured from the start of the transect line you drew above)

> **Optional:** Set transaction extraction mode to `'width'` and re-run the cell below to measure the width between two adjacent sets of coastlines (e.g. across the neck of a tombolo or sandbar). 

> **Optional:** Upload a GeoJSON or ESRI Shapefile containing transects using the `Upload` button to automatically extract shoreline positions from multiple coastal transects (uploaded files should be less than 5 megabytes in size).

In [9]:
deacoastlines.transect_app()

transect_app(children=(GridspecLayout(children=(HTML(value='<h3>Digital Earth Africa Coastlines shoreline tran…

The resulting figure shows how shorelines at this location have changed over time.

In `distance` mode, the y-axis shows the distance from the start of the transect to the location at which the transect intersects with each annual shoreline (shown on the x-axis).
If transects were drawn with a starting point on land, decreasing distances over time indicate the coastline at this location was retreating or eroding since 2000.
Conversely, increasing distances over time indicate the coastline was growing.

In `width` mode, the y-axis shows the distance (i.e. width) between the first and last time the transect intersects each annual shoreline. Increasing widths indicate that a sandbar, spit or peninsular was growing wider over time; decreasing widths indicate that it was eroding or growing narrower.

## Next steps: coastline erosion notebook

The [Coastal erosion notebook](../Real_world_examples/Coastal_erosion.ipynb) in this repository provides a simplified example of the method used to extract DE Africa Coastlines annual shoreline data. 
Run this notebook if you would like to generate more customised coastlines for a specific location, time period, epoch (e.g. annual or biennial coastlines) or tidal range (e.g. the position of the coastline at low, mid  or high tide).

> **Note:** This notebook currently supports extracting coastline data only, not generating rates of change statistics like those included in the DE Africa Coastlines product.

---

## Additional information

**License:** The code in this notebook is licensed under the [Apache License, Version 2.0](https://www.apache.org/licenses/LICENSE-2.0). 
Digital Earth Africa data is licensed under the [Creative Commons by Attribution 4.0](https://creativecommons.org/licenses/by/4.0/) license.

**Contact:** If you need assistance, please post a question on the [Open Data Cube Slack channel](http://slack.opendatacube.org/) or on the [GIS Stack Exchange](https://gis.stackexchange.com/questions/ask?tags=open-data-cube) using the `open-data-cube` tag (you can view previously asked questions [here](https://gis.stackexchange.com/questions/tagged/open-data-cube)).
If you would like to repoart an issue with this notebook, you can file one on [Github](https://github.com/digitalearthafrica/deafrica-sandbox-notebooks).

**Compatible datacube version:**

In [10]:
import datacube
print(datacube.__version__)

1.8.12


**Last Tested:**

In [11]:
from datetime import datetime
datetime.today().strftime('%Y-%m-%d')

'2023-03-30'